In [1]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Distributions

In [2]:
data = CSV.read("dataclean_imputed.csv", DataFrame);

In [3]:
X = Matrix(select(data, Not([:Column1, :id, :time, :status, :trt])));

In [4]:
#Define constants

#Number of patients
N = size(data)[1]

#Min _N and Max N_ number of patients allowed in an interpretable subset
_N = floor(0.1 * N)
N_ = ceil(0.3 * N)

94.0

In [5]:
#Define a list of patient numbers that are in each of the two treatment groups
#Note that 2 in the data set means placebo, and 1 means experimental group

T0 = findall(data[!,"trt"].==2) #indices of placebo patients
T1 = findall(data[!,"trt"].==1) #indices of experimental patients

T = [T0, T1]

2-element Vector{Vector{Int64}}:
 [5, 6, 7, 8, 10, 11, 12, 13, 14, 16  …  296, 298, 300, 301, 303, 305, 306, 307, 309, 312]
 [1, 2, 3, 4, 9, 15, 18, 19, 22, 24  …  291, 294, 295, 297, 299, 302, 304, 308, 310, 311]

In [6]:
#Patient-wise (from patient i=N) whether the patient is in placebo (1) or experimental group (2)
Ti = (data[!,"trt"].==1).+1;

In [7]:
function get_value_for_cut(s, k)
    # get max of X for feature s
    max_s = maximum(X[:, s])
    # get min of X for feature s
    min_s = minimum(X[:, s])
    # get cut value
    return min_s + (max_s - min_s) * (k - 1) / (Ks[s] - 1) 
end

get_value_for_cut (generic function with 1 method)

In [8]:
# find the cuts k for variable i in feature s for which X[i,s] is is smaller than the k-th cut
function get_k_L(i, s)
    cuts = []
    for k in 1:Ks[s]
        if X[i, s] < get_value_for_cut(s, k)
            push!(cuts, k)
        end
    end
    return cuts
end

# find the cuts k for variable i in feature s for which X[i,s] is is larger than the k-th cut
function get_k_U(i, s)
    cuts = []
    for k in 1:Ks[s]
        if X[i, s] > get_value_for_cut(s, k)
            push!(cuts, k)
        end
    end
    return cuts
end

# find the patients i for cut k in feature s for which X[i,s] is is smaller than the k-th cut
function get_i_L(s, k)
    patients = []
    for i in 1:N
        if X[i, s] < get_value_for_cut(s, k)
            push!(patients, i)
        end
    end
    return patients
end

# find the patients i for cut k in feature s for which X[i,s] is is larger than the k-th cut
function get_i_U(s, k)
    patients = []
    for i in 1:N
        if X[i, s] > get_value_for_cut(s, k)
            push!(patients, i)
        end
    end
    return patients
end

get_i_U (generic function with 1 method)

In [9]:
# treatment effect
# set multiplier to 10 for survival, 3 for transplant and 1 for death
function multiplier(patient)
    if data[patient, :status] == 2
        return 1
    elseif data[patient, :status] == 1
        return 3
    else
        return 10
    end
end

# define treatment effect as time * multiplier for each patient
v = [data[patient, :time] * multiplier(patient) for patient in 1:N];

In [10]:
# get patients for each :status
df_0 = data[data[!,:status].==0,:]
df_1 = data[data[!,:status].==1,:]
df_2 = data[data[!,:status].==2,:]

# for each patient group, cap the :time values to mean +- 3 * std
df_0[!,:time] = min.(df_0[!,:time], mean(df_0[!,:time]) + 3 * std(df_0[!,:time]))
df_0[!,:time] = max.(df_0[!,:time], mean(df_0[!,:time]) - 3 * std(df_0[!,:time]))
df_1[!,:time] = min.(df_1[!,:time], mean(df_1[!,:time]) + 3 * std(df_1[!,:time]))
df_1[!,:time] = max.(df_1[!,:time], mean(df_1[!,:time]) - 3 * std(df_1[!,:time]))
df_2[!,:time] = min.(df_2[!,:time], mean(df_2[!,:time]) + 3 * std(df_2[!,:time]))
df_2[!,:time] = max.(df_2[!,:time], mean(df_2[!,:time]) - 3 * std(df_2[!,:time]))

# for each patient group replace :time with the min-max scaled value
df_0[!,:time] = (df_0[!,:time] .- minimum(df_0[!,:time])) ./ (maximum(df_0[!,:time]) - minimum(df_0[!,:time]))
df_1[!,:time] = (df_1[!,:time] .- minimum(df_1[!,:time])) ./ (maximum(df_1[!,:time]) - minimum(df_1[!,:time]))
df_2[!,:time] = (df_2[!,:time] .- minimum(df_2[!,:time])) ./ (maximum(df_2[!,:time]) - minimum(df_2[!,:time]))

# join the dataframes again
data = vcat(df_0, df_1, df_2)

# sort dataframe by :id
data = sort(data, :id)

# define treatment effect as the adjusted time for each patient
v = data[!,:time];

In [11]:
function get_features()
    L_opt = value.(L);
    U_opt = value.(U);
    # get the indices of the lower bound cuts 
    # for each feature
    L_ind = [findall(L_opt[s,:] .== 1)[1] for s=1:S]

    # get the indices of the upper bound cuts
    # for each feature
    U_ind = [findall(U_opt[s,:] .== 1)[1] for s=1:S];

    # find features for which the lower bound is not the minimum
    # and the upper bound is not the maximum
    q_ind = findall(value.(q) .== 1)

    # get the value of the lower and upper bound cuts for features in q_ind
    L_val = [get_value_for_cut(s, L_ind[s]) for s in q_ind]
    U_val = [get_value_for_cut(s, U_ind[s]) for s in q_ind];

    # get names of features in q_ind
    q_names = [names(data)[s+5] for s in q_ind]

    str = "Features that make up the interpretable subgroup are: " * join(q_names, ", ") *
            "\nThe subgroup has the following bounds:"
    # print the lower and upper bound cuts for features in q_ind
    for i=1:length(q_ind)
        str *= "\n" * string(q_names[i]) * " ∈ [" * string(L_val[i]) * ", " * string(U_val[i]) * "]"
    end
    str *= "\n--------------------------------------------------------"
    return str
end

get_features (generic function with 1 method)

In [12]:
function get_ATE()
    # get patients in the interpretable subgroup
    z_ind = findall(value.(z) .== 1)
    # get patients in the interpretable subgroup from treatment group 1
    z1_ind = intersect(z_ind, T1)

    # get average treatment effect for patients in the interpretable subgroup
    ATE = mean(v[z1_ind])

    # get patients in the interpretable subgroup from treatment group 0
    z0_ind = intersect(z_ind, T0)

    # get average treatment effect for patients in the interpretable subgroup
    ATE_not = mean(v[z0_ind])

    # print ATE and ATE_not
    str = "ATE: " * string(ATE) * 
        "\nATE_not: " * string(ATE_not) * 
        "\nDifference: " * string(ATE - ATE_not) *
        "\nPercentage difference: " * string(round((ATE - ATE_not) / ATE_not * 100, digits=2)) * "%"
    str *= "\n--------------------------------------------------------"
    return str
end

get_ATE (generic function with 1 method)

In [22]:
S = size(X)[2]
K = 15
S_0 = 4
rho = 0.2;

In [14]:
# cut ranges for variables (make them start at 0)
Ks = [K, 3, 3, 3, 3, 4, K, K, K, K, K, K, K, K, K, 7];

In [23]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1)
set_optimizer_attribute(model, "Threads", 20)
#set_optimizer_attribute(model, "MIPGap", 0.005)
set_optimizer_attribute(model, "TimeLimit", 36000)

# variables
#Variables
@variable(model, z[1:N], Bin) #Indicator variable - if each patient i is contained within the interpretable subgroup
@variable(model, theta[_N:N_, 1:2], Bin) #Indicator variable - if j between _N and N_ is equal to the number of patients from treatment group t within the interpretable subgroup
@variable(model, zeta[1:N,_N:N_]) #Indicator variable that is 1 iff both zi = 1 and thetaj = 1
@variable(model, L[s=1:S, k=1:maximum(Ks)], Bin) # Lower bound indicator variable, 1 iff cut k is the lower bound for feature s in the interpretable subgroup
@variable(model, U[s=1:S, k=1:maximum(Ks)], Bin) # Upper bound indicator variable, 1 iff cut k is the upper bound for feature s in the interpretable subgroup
@variable(model, q[s=1:S], Bin) # Indicator variable, 1 iff feature s has a non-extremal lower and/or upper bound in the interpretable subgroup

# constraints
@constraint(model, [i=1:N], z[i] + sum(sum(L[s, k] for k in get_k_L(i, s)) + sum(U[s, k] for k in get_k_U(i, s)) for s=1:S) >= 1)

@constraint(model, [s=1:S, k=1:Ks[s], i in get_i_L(s, k)], z[i] + L[s, k] <= 1)
@constraint(model, [s=1:S, k=1:Ks[s], i in get_i_U(s, k)], z[i] + U[s, k] <= 1)

@constraint(model, [s=1:S], sum(L[s, k] for k=1:Ks[s]) == 1) # We can only ever select one lower bound per feature
@constraint(model, [s=1:S], sum(U[s, k] for k=1:Ks[s]) == 1) # We can only ever select one upper bound per feature

# @constraint(model, [s=1:S], sum(L[s, k] for k=1:K) == 1)
# @constraint(model, [s=1:S], sum(U[s, k] for k=1:K) == 1)

@constraint(model, [s=1:S], q[s] + L[s, 1] >= 1) # Either the lower bound is not the extremal value, or the feature is not selected
@constraint(model, [s=1:S], q[s] + U[s, Ks[s]] >= 1) # Either the upper bound is not the extremal value, or the feature is not selected
@constraint(model, [s=1:S], q[s] + L[s, 1] + U[s, Ks[s]] <= 2) # The feature can only be selected if at most one of the bounds is the extremal value

@constraint(model, sum(q[s] for s=1:S) <= S_0) # Select at most S_0 features in the interpretable subgroup

@constraint(model, [t=1:2], _N <= sum(z[i] for i in T[t]) <= N_) # The number of patients within the interpretable subgroup from EACH treatment group must be within the bounds _N and N_

@constraint(model, [i=1:N, j=_N:N_], zeta[i,j] <= theta[j,Ti[i]]) # Ensuring z works as indicator variable (see variable section)
@constraint(model, [i=1:N, j=_N:N_], zeta[i,j] <= z[i])
@constraint(model, [i=1:N, j=_N:N_], zeta[i,j] >= theta[j,Ti[i]] + z[i] - 1)

@constraint(model, [t=1:2], sum(sum((1/j)*zeta[i,j] for j in _N:N_) for i in T[t]) == 1) # Confirming that the sum of the patients in the interpretable cluster equals j for each treatment group

@constraint(model, [t=1:2], sum(theta[j,t] for j=_N:N_) == 1) # Ensuring theta works as indicator variable (see variable section)

@constraint(model, [i=1:N, j=_N:N_], 0<=zeta[i,j]<=1) # Zeta bounds

@constraint(model, sum(z[i] for i in z_1_idx) <= rho * (length(z_1_idx) + sum(z[i] for i in z_0_idx))) # Ensuring that the interpretable subgroup contains at most rho * (N_1 + N_0) patients from treatment group 1

@objective(model, Max, sum(sum((1/j) * v[i] * zeta[i,j] for j in _N:N_) for i in T1) - sum(sum((1/j) * v[i] * zeta[i,j] for j in _N:N_) for i in T0)); # Objective function

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-08
Set parameter Threads to value 20
Set parameter TimeLimit to value 36000


In [24]:
optimize!(model)

In [ ]:
str = get_features() * "\n" * get_ATE() * "\nPatients in the interpretable subgroup: " * string(sum(value.(z)))
println(str)

Features that make up the interpretable subgroup are: chol, copper, ast, protime
The subgroup has the following bounds:
chol ∈ [356.42857142857144, 947.5]
copper ∈ [45.714285714285715, 588.0]
ast ∈ [57.12857142857143, 303.3571428571429]
protime ∈ [9.57857142857143, 15.364285714285716]
--------------------------------------------------------
ATE: 0.4400028870610881
ATE_not: 0.234502340572481
Difference: 0.20550054648860708
Percentage difference: 87.63%
--------------------------------------------------------
Patients in the interpretable subgroup: 78.0


In [15]:
# load results
path = "K=$K/S_0=$S_0"
# L = Matrix(CSV.read("$path/L_opt.csv", DataFrame))
# U = Matrix(CSV.read("$path/U_opt.csv", DataFrame))
# q = CSV.read("$path/q_opt.csv", DataFrame)[!, :q]
z_1 = CSV.read("$path/z_opt.csv", DataFrame)[!, :z];

In [19]:
# get indices of z_1 that are 1
z_1_idx = findall(x -> x == 1, z_1);

# get indices of z_1 that are 0
z_0_idx = findall(x -> x == 0, z_1);

In [ ]:
# save optimal z, L, U and q to disk
# create dataframe for each
z_opt = DataFrame(z=value.(z))
L_opt = DataFrame(value.(L), :auto)
U_opt = DataFrame(value.(U), :auto)
q_opt = DataFrame(q=value.(q))

# save to disk
# create directory if it doesn't exist
isdir("K=$K") || mkdir("K=$K")
isdir("K=$K/S_0=$S_0") || mkdir("K=$K/S_0=$S_0")
CSV.write("K=$K/S_0=$S_0/z_opt_2.csv", z_opt)
CSV.write("K=$K/S_0=$S_0/L_opt_2.csv", L_opt)
CSV.write("K=$K/S_0=$S_0/U_opt_2.csv", U_opt)
CSV.write("K=$K/S_0=$S_0/q_opt_2.csv", q_opt)

# write str to file
open("K=$K/S_0=$S_0/results_2.txt", "w") do io
    write(io, str)
end;

In [154]:
# average :time of data per :status
function average_time_per_status(status)
    return mean(data[data[:, :status] .== status, :time])
end

# for each status get the average :time
average_times = [average_time_per_status(status) for status in 0:2]

mean_time = mean(data[:, :time])

1979.1666666666667

In [155]:
average_times

3-element Vector{Float64}:
 2391.78231292517
 1511.611111111111
 1508.5495495495495

In [157]:
# max :time of data per :status
function max_time_per_status(status)
    return maximum(data[data[:, :status] .== status, :time])
end

# for each status get the max :time
max_times = [max_time_per_status(status) for status in 0:2]

3-element Vector{Int64}:
 4556
 3092
 4191

In [156]:
# average age
mean_age = mean(data[:, :age])

49.799660744576386